In [1]:
import os

api = os.getenv("OPEN_ROUTER_API")

In [2]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(
  openai_api_key=api,
  openai_api_base="https://openrouter.ai/api/v1",
  model_name="deepseek/deepseek-r1-distill-llama-70b:free"
  """model_kwargs={
    "headers": {
      "HTTP-Referer": getenv("YOUR_SITE_URL"),
      "X-Title": getenv("YOUR_SITE_NAME"),
    }
  },""",
  temperature=0
)


C:\Users\Rafiq\AppData\Local\Temp\ipykernel_14708\447765346.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [3]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# <center> Home made tool calling </center>

In [5]:
#Let's start by defining a bunch of data analysis functions : 

import numpy as np

def compute_mean(column: str) -> float:
    """Compute the mean of a column in the Titanic dataset."""
    return np.mean(df[column])

def compute_median(column: str) -> float:
    """Compute the median of a column in the Titanic dataset."""
    return np.median(df[column])

def compute_std(column: str) -> float:
    """Compute the standard deviation of a column in the Titanic dataset."""
    return np.std(df[column])

def compute_max(column: str) -> float:  
    """Compute the maximum of a column in the Titanic dataset."""
    return np.max(df[column])

def compute_min(column: str) -> float:
    """Compute the minimum of a column in the Titanic dataset."""
    return np.min(df[column])   

def compute_count(column: str) -> int:
    """Compute the count of non-null values in a column in the Titanic dataset."""
    return df[column].count()

def compute_unique(column: str) -> list:
    """Compute the unique values in a column in the Titanic dataset."""
    return df[column].unique().tolist()

In [ ]:
#Let's create some user examples to enmbed : 

mean_sentences = [
    "What is the mean of the column \"Age\" in the dataset?",
    "Could you calculate the average value within the \"Age\" field?",
    "What's the arithmetic mean hiding in our \"Age\" variable?",
    "I'm curious about the expected value lurking in the \"Age\" attribute.",
    "Please compute the central tendency of the \"Age\" feature.",
    "What numerical center emerges from our \"Age\" dimension?",
    "Calculate the typical value represented in the \"Age\" metric."
]

median_sentences = [
    "What's the median value of the \"Age\" column?",
    "Could you find the middle point in our \"Age\" distribution?",
    "What's the 50th percentile value sitting in the \"Age\" field?",
    "What central figure divides our \"Age\" data into equal halves?",
    "Please identify the midpoint value from the \"Age\" attribute.",
    "What number creates perfect balance in our \"Age\" column?",
    "Calculate the value that splits the \"Age\" data in half."
]

std_dev_sentences = [
    "What's the standard deviation of the \"Age\" column?",
    "How much dispersion exists across the \"Age\" field?",
    "Calculate the spread parameter for the \"Age\" variable.",
    "What's the square root of variance in our \"Age\" metric?",
    "Quantify the typical distance from the mean in our \"Age\" data.",
    "How much does the \"Age\" data typically fluctuate from center?",
    "What's the measure of variability within the \"Age\" attribute?"
]

min_sentences = [
    "What's the minimum value in the \"Age\" column?",
    "What's the lowest reading found in the \"Age\" field?",
    "What's the smallest figure recorded in the \"Age\" attribute?",
    "What's the bottom value sitting in our \"Age\" variable?",
    "Please identify the floor value from the \"Age\" dimension.",
    "What's the least measurement in our \"Age\" metric?",
    "What absolute lower bound exists in the \"Age\" collection?"
]

max_sentences = [
    "What's the maximum value in the \"Age\" column?",
    "What's the highest measurement in our \"Age\" dataset?",
    "What peak figure appears in the \"Age\" variable?",
    "What's the ceiling value within the \"Age\" attribute?",
    "Please identify the supreme reading from our \"Age\" metric.",
    "What topmost measurement exists in our \"Age\" field?",
    "What's the ultimate value in our \"Age\" dimension?"
]

non_null_count_sentences = [
    "What's the count of non-null values in the \"Age\" column?",
    "How many populated entries exist in the \"Age\" field?",
    "What's the tally of present values in our \"Age\" attribute?",
    "How many actual datapoints live in the \"Age\" dimension?",
    "Calculate the quantity of non-missing records in our \"Age\" variable.",
    "What's the census of populated cells in the \"Age\" metric?",
    "How large is the collection of existing values in our \"Age\" registry?"
]

unique_values_sentences = [
    "What's the unique value list for the \"Age\" column?",
    "Show me the distinct entries found in the \"Age\" field.",
    "What singular values comprise our \"Age\" attribute?",
    "Enumerate the non-repeating items in our \"Age\" variable.",
    "List the exclusive members within our \"Age\" dimension.",
    "What's the catalog of individual values in our \"Age\" metric?",
    "Display the one-of-a-kind entries from our \"Age\" registry."
]

#This approach gave us such results : Total tests: 60, Passed: 0.6833333333333333, Failed: 0.31666666666666665
#Creating a virtual cluster with multiple expamples of the same function made them overlap. And max failed - for this approach rather try a mean or median (Find the closest cluster, not closest point)

In [ ]:
synonym_string = [
    #0
    "mean, average, mean value, arithmetic mean, typical value, expected value, central value, central tendency, average out to, overall mean, mean of the data, ",
    #1
    "median, median value, middle value, 50th percentile, midpoint, value in the center, middlemost value, halfway value, center point, ",
    #2
    "max, maximum, maximum value, highest value, largest number, top value, peak, upper bound, highest recorded value, extreme value, greatest value, ",
    #3
    "standard deviation, std, variability, spread, dispersion, statistical spread, deviation from the mean, volatility, fluctuation, consistency measure, ",
    #4
    "count, number of values, total entries, non-null count, filled rows, non-missing values, number of observations, row count, valid entries, total data points, data count, ",
    #5
    "unique, unique values, distinct values, different entries, non-repeating values, unique items, all different values, value set, distinct numbers, set of unique entries, diversity in values",
    #6
    "min, minimum, minimum value, lowest value, smallest number, bottom value, floor, lower bound, least value, lowest recorded value, "
]

#let's use "blackholes" approach, basically very precise points in space well in the middle of each function definition meaning
#This approach has better results : Total tests: 60, Passed: 0.8, Failed: 0.2

In [ ]:
#Last approach : synonym - sentence - inclusion. We'll move the coordinates of the embedding space a bit by wrapping the synonyms into a sentence, to fall closer to the user input.

synonym_string = [
    #0
    """What is the "mean, average, mean value, arithmetic mean, typical value, expected value, central value, central tendency, average out to, overall mean, mean of the data " Of the column "Age" in the dataset?""",
    #1
    """What is the "median, median value, middle value, 50th percentile, midpoint, value in the center, middlemost value, halfway value, center point" Of the column "Age" in the dataset?""",
    #2
    """What is the "max, maximum, maximum value, highest value, largest number, top value, peak, upper bound, highest recorded value, extreme value, greatest value" Of the column "Age" in the dataset?""",
    #3
    """What is the "standard deviation, std, variability, spread, dispersion, statistical spread, deviation from the mean, volatility, fluctuation, consistency measure" Of the column "Age" in the dataset?""",
    #4
    """What is the "count, number of values, total entries, non-null count, filled rows, non-missing values, number of observations, row count, valid entries, total data points, data count" Of the column "Age" in the dataset?""",
    #5
    """What is the "unique, unique values, distinct values, different entries, non-repeating values, unique items, all different values, value set, distinct numbers, set of unique entries, diversity in values" Of the column "Sex" in the dataset?""",
    #6
    """What is the "min, minimum, minimum value, lowest value, smallest number, bottom value, floor, lower bound, least value, lowest recorded value" Of the column "Age" in the dataset?"""
]

#Results : Total tests: 60, Passed: 0.85, Failed: 0.15

In [ ]:
import inspect

# Let's create a dictionary with the function definitions as strings, we'll also give then an example of how to call them in the prompt.
function_definitions = {
    "compute_mean": [inspect.getsource(compute_mean), """user: What is the mean of the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_mean column:Age""", synonym_string[0]],
    "compute_median": [inspect.getsource(compute_median), """user: What is the median of the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_median column:Age""", synonym_string[1]],
    "compute_std": [inspect.getsource(compute_std), """user: What is the standard deviation of the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_std column:Age""", synonym_string[3]],
    "compute_max": [inspect.getsource(compute_max), """user: What is the max of the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_max column:Age""", synonym_string[2]],
    "compute_min": [inspect.getsource(compute_min), """user: What is the min of the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_min column:Age""", synonym_string[6]],
    "compute_count": [inspect.getsource(compute_count), """user: What is the count in the column "Age" in the dataset? \n data scientist: ASKARABADINGDONG compute_count column:Age""", synonym_string[4]],
    "compute_unique": [inspect.getsource(compute_unique), """user: What are the unique values in the column "Sex" in the dataset? \n data scientist: ASKARABADINGDONG compute_unique column:Sex""", synonym_string[5]],
}

In [10]:
from sentence_transformers import SentenceTransformer

# Your custom folder path
custom_model_folder = r"C:\embeddingModel"

# Load and cache model to custom folder
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder=custom_model_folder)

c:\Users\Rafiq\anaconda3-updated\envs\anaconda_llama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#Let's create an embedding for the user's question, the "...sentences" variables exemple so we can get the most relevant function to call.
embedding_list = []
for value in function_definitions.values():
    embedding = model.encode(value[2])
    embedding_list.append(embedding)
    value.append(embedding_list)
    embedding_list = []

In [59]:
#For each function, we have 7 setences to embed, so we have 7 embeddings for each function. For each function, for each embedding we need to compute the cosine similarity with the user question embedding
#Them for each function we need to get the max cosine similarity, and then we can get the function with the max cosine similarity.
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_most_similar_function(user_question: str) -> str:
    # Embed the user question
    user_question_embedding = model.encode(user_question)

    # Initialize variables to track the most similar function and its similarity score
    most_similar_function = None
    max_similarity = -1

    # Iterate through each function and its embeddings
    for function_name, value in function_definitions.items():
        # Compute cosine similarity for each embedding of the function
        similarities = cosine_similarity([user_question_embedding], value[3])
        max_function_similarity = np.max(similarities)

        # Update if this function is more similar than the previous best
        if max_function_similarity > max_similarity:
            max_similarity = max_function_similarity
            most_similar_function = function_name

    return most_similar_function

In [12]:
#Only one sentence per function
def get_most_relevant_function(user_question: str) -> str:
    """Get the most relevant function to call based on the user's question."""
    user_embedding = model.encode(user_question)
    similarities = []
    for value in function_definitions.values():
        similarity = cosine_similarity([user_embedding], [value[3][0]])[0][0]
        similarities.append(similarity)
    most_relevant_function = list(function_definitions.keys())[np.argmax(similarities)]
    return most_relevant_function

In [13]:
#Let's try it out :
tests = [
    # compute_max
    ["Find the highest value in 'Salary'", "compute_max"],
    ["What’s the maximum in 'Height'", "compute_max"],
    ["Give me the max of 'Exam Score'", "compute_max"],
    ["Top value in 'Temperature'", "compute_max"],
    ["Largest number in the column 'Revenue'", "compute_max"],
    ["Highest recorded value for 'Speed'", "compute_max"],
    ["Show the peak value from 'Stock Price'", "compute_max"],
    ["What is the upper bound of 'Age'", "compute_max"],
    ["What's the extreme high value in 'Distance'", "compute_max"],
    ["Get the greatest value in 'Response Time'", "compute_max"],

    # compute_mean
    ["Calculate the average for 'Salary'", "compute_mean"],
    ["What’s the mean of 'Height'", "compute_mean"],
    ["Give me the average of the column 'Exam Score'", "compute_mean"],
    ["Typical value in 'Temperature'", "compute_mean"],
    ["Find the arithmetic mean for 'Revenue'", "compute_mean"],
    ["What is the expected value in 'Speed'", "compute_mean"],
    ["On average, what is the 'Stock Price'?", "compute_mean"],
    ["Compute the central tendency (mean) for 'Age'", "compute_mean"],
    ["What does 'Distance' average out to?", "compute_mean"],
    ["Show me the mean value of 'Response Time'", "compute_mean"],

    # compute_median
    ["Get the median of 'Salary'", "compute_median"],
    ["Middle value in 'Height'", "compute_median"],
    ["What’s the central value in 'Exam Score'?", "compute_median"],
    ["Find the 50th percentile of 'Temperature'", "compute_median"],
    ["What’s the value halfway through 'Revenue'?", "compute_median"],
    ["Compute the median value of the 'Speed' column", "compute_median"],
    ["Show the central tendency (median) of 'Stock Price'", "compute_median"],
    ["What’s the value that splits 'Age' in half?", "compute_median"],
    ["Get the midpoint value for 'Distance'", "compute_median"],
    ["Return the center value in 'Response Time'", "compute_median"],

    # compute_std
    ["How much does 'Salary' vary?", "compute_std"],
    ["What is the standard deviation of 'Height'?", "compute_std"],
    ["Find the spread of 'Exam Score'", "compute_std"],
    ["Compute how dispersed 'Temperature' values are", "compute_std"],
    ["Measure the variability in 'Revenue'", "compute_std"],
    ["How consistent is 'Speed'?", "compute_std"],
    ["Get the deviation from the mean for 'Stock Price'", "compute_std"],
    ["Show me the standard deviation in the 'Age' column", "compute_std"],
    ["What’s the volatility of 'Distance'?", "compute_std"],
    ["How much fluctuation is in 'Response Time'?", "compute_std"],

    # compute_count
    ["How many values are there in 'Salary'?", "compute_count"],
    ["Count the non-null entries in 'Height'", "compute_count"],
    ["Give me the number of filled rows in 'Exam Score'", "compute_count"],
    ["What’s the size of the 'Temperature' column?", "compute_count"],
    ["Compute the number of valid entries in 'Revenue'", "compute_count"],
    ["How many observations are recorded in 'Speed'?", "compute_count"],
    ["Get the count of 'Stock Price'", "compute_count"],
    ["Total number of data points in 'Age'", "compute_count"],
    ["Show how many rows contain 'Distance'", "compute_count"],
    ["How many non-missing values exist in 'Response Time'?", "compute_count"],

    # compute_unique
    ["What are the unique values in 'Salary'?", "compute_unique"],
    ["List all distinct entries in 'Height'", "compute_unique"],
    ["Give me all the different values from 'Exam Score'", "compute_unique"],
    ["What are the unique items in 'Temperature'?", "compute_unique"],
    ["Show the set of unique values in the 'Revenue' column", "compute_unique"],
    ["What values appear at least once in 'Speed'?", "compute_unique"],
    ["Find the distinct numbers used in 'Stock Price'", "compute_unique"],
    ["List non-repeating values in 'Age'", "compute_unique"],
    ["Get the set of all unique data points in 'Distance'", "compute_unique"],
    ["Which values occur in 'Response Time'?", "compute_unique"]
]

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

passed = 0
failed = 0

for test in tests:
    user_question, expected_function = test
    most_relevant_function = get_most_relevant_function(user_question)
    if(most_relevant_function != expected_function): 
        print(f"Test failed for question: {user_question}")
        print(f"Expected: {expected_function}, Got: {most_relevant_function}")
        failed = failed + 1
    else:
        passed = passed + 1

Test failed for question: What’s the mean of 'Height'
Expected: compute_mean, Got: compute_max
Test failed for question: How much does 'Salary' vary?
Expected: compute_std, Got: compute_median
Test failed for question: Find the spread of 'Exam Score'
Expected: compute_std, Got: compute_median
Test failed for question: How many values are there in 'Salary'?
Expected: compute_count, Got: compute_median
Test failed for question: What’s the size of the 'Temperature' column?
Expected: compute_count, Got: compute_std
Test failed for question: Give me all the different values from 'Exam Score'
Expected: compute_unique, Got: compute_median
Test failed for question: What values appear at least once in 'Speed'?
Expected: compute_unique, Got: compute_min
Test failed for question: List non-repeating values in 'Age'
Expected: compute_unique, Got: compute_min
Test failed for question: Which values occur in 'Response Time'?
Expected: compute_unique, Got: compute_mean


In [16]:
#We're not there yet...
total = passed + failed
print(f"Total tests: {total}, Passed: {passed/total}, Failed: {failed/total}")

Total tests: 60, Passed: 0.85, Failed: 0.15


In [17]:
#What if we do a top n instead ? 

def get_top_n_functions(user_question: str, n=2) -> list:
    """Get the top N most relevant functions to call based on the user's question."""
    user_embedding = model.encode(user_question)
    similarities = []
    for value in function_definitions.values():
        similarity = cosine_similarity([user_embedding], [value[3][0]])[0][0]
        similarities.append(similarity)
    top_n_indices = np.argsort(similarities)[-n:][::-1]
    top_n_functions = [list(function_definitions.keys())[i] for i in top_n_indices]
    return top_n_functions

In [27]:
#let's do the test with top 2 : 

passed = 0
failed = 0

for test in tests:
    user_question, expected_function = test
    most_relevant_functions = get_top_n_functions(user_question, 5)
    if(str(expected_function) not in most_relevant_functions): 
        print(f"Test failed for question: {user_question}")
        print(f"Expected: {expected_function}, Got: {most_relevant_functions}")
        failed = failed + 1
    else:
        passed = passed + 1

Test failed for question: How much does 'Salary' vary?
Expected: compute_std, Got: ['compute_median', 'compute_max', 'compute_mean', 'compute_unique', 'compute_min']
Test failed for question: What’s the size of the 'Temperature' column?
Expected: compute_count, Got: ['compute_std', 'compute_median', 'compute_max', 'compute_min', 'compute_mean']
Test failed for question: Give me all the different values from 'Exam Score'
Expected: compute_unique, Got: ['compute_median', 'compute_mean', 'compute_max', 'compute_count', 'compute_min']
Test failed for question: Which values occur in 'Response Time'?
Expected: compute_unique, Got: ['compute_mean', 'compute_min', 'compute_max', 'compute_median', 'compute_count']


In [26]:
print(f"Total tests: {total}, Passed: {passed/total}, Failed: {failed/total}")

Total tests: 60, Passed: 0.9, Failed: 0.1


We can see that going for top 2 or top 3 doesn't improve the performance that much.
The embedding model still has trouble to understand "compute_unique" "and compute_count".

But that's OKAY. Even if the best function is not found the LLM can get it by intuition (or not actually)

In [ ]:
#Let's create a prompt template to generate the function calls

def prompt_function_call(n, function_list, function_definitions):
    """Generate a prompt for the LLM to call the most relevant function."""
    introduction = "You are a data scientist. You are capable of analyzing data and providing insights.\nYou also have tools to gain insights from data, the tools are python functions.\nBelow is the list of tools:"
    tools = -"-----start of tools------\n"
    for i in range(n):
        function_name = function_list[i]
        function_code = function_definitions[function_name][0]
        tools = tools + f"def {function_name}:\n{function_code}\n"
    tools = tools + "-------end of tools-------\n"
    prompt = introduction + tools + """If you feel like you need to use a tool, answer only by saying ASKARABADINGDONG and then the name of the tool you want to use and which arguments you want to pass to it,
    no explanation, no other text.\nExample: \n ----------start of examples----------"""

    for j in range(n):
        function_name = function_list[j]
        example = function_definitions[function_name][1]
        prompt = prompt + f"\n{example}\n"
    prompt = prompt + "----------end of examples----------\n"
    prompt = "Else, answer the question directly. Never say ASKARABADINGDONG if you don't need to use a tool.\n\nHere's the question from the user:" + str(user_question)
    return prompt


In [40]:
ai_message = llm.invoke(prompt)
print(ai_message)

content='ASKARABADINGDONG compute_mean column:Age' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 221, 'total_tokens': 333, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1-distill-llama-70b:freemodel_kwargs={\n    "headers": {\n      "HTTP-Referer": getenv("YOUR_SITE_URL"),\n      "X-Title": getenv("YOUR_SITE_NAME"),\n    }\n  },', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-68724325-d381-419d-9307-d864de3f7aeb-0'


In [ ]:
import inspect
import numpy as np
def compute_mean(column: str) -> float:
    """Compute the mean of a column in the Titanic dataset."""
    return np.mean(df[column])

if (ai_message.content.__contains__("ASKARABADINGDONG")):
    # Extract the function name and arguments from the AI message
    function_call = ai_message.content.split(" ")[1]
    
    #get function to call
    fnToCall = globals()[function_call]
    #get function arguments
    
    #get function signature
    signature = inspect.signature(fnToCall)
    #get function arguments
    fnArgs = list(signature.parameters.keys())
    #in this exemple there is only one argument, so we can get it directly
    arg = fnArgs[0]
    #get the value of the argument from the AI message
    argValue = ai_message.content.split(arg + ":")[1].split(" ")[0]
    #call the function with the argument value
    result = fnToCall(argValue)
    #print the result
    print(f"The mean of the column {arg} is {result}")

The mean of the column column is 29.69911764705882


In [42]:
np.mean(df["Age"])

np.float64(29.69911764705882)

In [34]:
print(list(signature.parameters.keys()))

['column']


In [14]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool
import numpy as np

#Define inputs as a class
class MeanInput(BaseModel):
    column: str = Field(description="The column to calculate the mean of", exemple="age")

#Use predefined schema 
@tool("compute_mean", args_schema=MeanInput)
def compute_mean(column: str) -> float:
    """Compute the mean of a column in the Titanic dataset."""
    return np.mean(df[column])

#bind the tool to the llm
llm_with_tools = llm.bind_tools(
    tools=[compute_mean],
    tool_choice={"type": "function", "function": {"name": "compute_mean"}},
)

NotImplementedError: 